Imports

In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import folium
import csv

Cargar archivos requerido por la instancia

In [2]:
clients_df = pd.read_csv("Proyecto_Caso_Base/clients.csv")
depots_df = pd.read_csv("Proyecto_Caso_Base/depots.csv")
vehicles_df = pd.read_csv("Proyecto_Caso_Base/vehicles.csv")


In [3]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())

   ClientID  LocationID  Demand  Longitude  Latitude
0         1           2      13 -74.098938  4.597954
1         2           3      15 -74.075571  4.687821
2         3           4      12 -74.107085  4.709494
3         4           5      15 -74.097280  4.605029
4         5           6      20 -74.164641  4.648464
   LocationID  DepotID  Longitude  Latitude  Capacity
0           1        1 -74.153536  4.743359        90
  VehicleType  Capacity  Range
0     Gas Car       130    170
1     Gas Car       140    200
2     Gas Car       120    180
3     Gas Car       100     90
4     Gas Car        70    100


In [4]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))

In [5]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)

Conjunto C: [1]
Capacidades CAP_i: {1: 90}


In [6]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Demand"])) 

In [7]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Capacidades Demanda: {1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15, 11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12, 19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11}


Distancia harvesiana

In [8]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c

In [9]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)

In [10]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 17.26
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 10.62
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 6.38
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 16.6
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 10.62


In [11]:
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))

In [12]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)

Vehículos: [0, 1, 2, 3, 4, 5, 6, 7]
Capacidades: {0: 130, 1: 140, 2: 120, 3: 100, 4: 70, 5: 55, 6: 110, 7: 114}
Rangos: {0: 170, 1: 200, 2: 180, 3: 90, 4: 100, 5: 170, 6: 150, 7: 140}


In [13]:
for v in V:
    print(f"vehiculo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

vehiculo 0: Tipo=Gas Car, Capacidad=130, Rango=170
vehiculo 1: Tipo=Gas Car, Capacidad=140, Rango=200
vehiculo 2: Tipo=Gas Car, Capacidad=120, Rango=180
vehiculo 3: Tipo=Gas Car, Capacidad=100, Rango=90
vehiculo 4: Tipo=Gas Car, Capacidad=70, Rango=100
vehiculo 5: Tipo=Gas Car, Capacidad=55, Rango=170
vehiculo 6: Tipo=Gas Car, Capacidad=110, Rango=150
vehiculo 7: Tipo=Gas Car, Capacidad=114, Rango=140
